In [1]:
!pip install python-terrier --quiet
!pip install nltk --quiet
!pip install --ignore-installed blinker
!pip install git+https://github.com/experimaestro/experimaestro-ir.git --quiet
!pip install transformers
!pip install flair
!git clone https://github.com/terrierteam/terrier-prf/
!apt-get install maven   #used for Java projects to manage project dependencies and build processes
%cd /content/terrier-prf/
!mvn install
!pwd
%cd ..

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.9/337.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.9 MB/s eta 0:00:00
  Preparing metadata (setup.

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import *
from nltk.stem.porter import *
from flair.data import Sentence
from flair.embeddings import WordEmbeddings
from transformers import AutoTokenizer, AutoModel
import tensorflow as tf
import tensorflow_hub as hub
import gensim
from gensim.models import Word2Vec
pd.set_option('display.max_colwidth', 150)
import pyterrier as pt
if not pt.started():
  # In this lab, we need to specify that we start PyTerrier with PRF enabled
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

terrier-assemblies 5.9 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8



In [3]:
data = pd.read_csv("/content/emotion_sentimen_dataset.csv")
data

,ID,text,Emotion
0,0,i seriously hate one subject to death but now i feel reluctant to drop it,hate
1,1,im so full of life i feel appalled,neutral
2,2,i sit here to write i start to dig out my feelings and i think that i am afraid to accept the possibility that he might not make it,neutral
3,3,ive been really angry with r and i feel like an idiot for trusting him in the first place,anger
4,4,i feel suspicious if there is no one outside like the rapture has happened or something,neutral
...,...,...,...
83677,83677,i began to feel pretty crappy,neutral
83678,83678,i say so because i have always considered uncle paul as that darling uncle with a knack for telling jokes exciting smiles and laughter and for mak...,fun
83679,83679,i feel lonely and unloved even though i was the one that ended the relationship,love
83680,83680,i have a sinking feeling there are more than a few people who have entertained the same heretical thought that i have,neutral


In [4]:
nltk.download('stopwords')
print(stopwords.words('english'))
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
stop_words = set(stopwords.words('english'))

In [6]:
# Function to remove stopwords
def remove_stopwords(text):

    tokens = word_tokenize(text)
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in stop_words] #Lower is used to normalize al the words make them in lower case
    print('Tokens are:',tokens,'\n')
    return ' '.join(filtered_tokens)

# Apply the remove_stopwords function to the 'content' column
data['processed_text'] = data['text'].apply(remove_stopwords)

Streaming output truncated to the last 5000 lines.
Tokens are: ['i', 'feel', 'helpless', 'beng', 'unable', 'to', 'reeally', 'help', 'them'] 

Tokens are: ['i', 'could', 'not', 'help', 'feeling', 'thatrupert', 'meant', 'to', 'be', 'rude', 'to', 'my', 'father', 'though', 'his', 'words', 'were', 'quite', 'polite'] 

Tokens are: ['i', 'feel', 'soooo', 'cool', 'i', 'would', 'be', 'free', 'to', 'post', 'pictures', 'and', 'not', 'feel', 'like', 'an', 'idiot', 'pictures', 'on', 'blogs'] 

Tokens are: ['i', 'get', 'the', 'feeling', 'with', 'all', 'these', 'super', 'couponing', 'shows', 'out', 'there', 'and', 'some', 'of', 'those', 'savvy', 'shoppers', 'that', 'buy', 'identical', 'items', 'and', 'pay', 'two', 'cents', 'total', 'for', 'it', 'companies', 'are', 'being', 'a', 'little', 'more', 'cautious', 'with', 'the', 'coupons', 'they', 'give', 'out'] 

Tokens are: ['i', 'always', 'feel', 'skeptical', 'about', 'how', 'sincere', 'new', 'year', 's', 'resolutions', 'are'] 

Tokens are: ['im', 'feeli

In [7]:
stemmer = PorterStemmer()

In [8]:
def Steem_text(text):

    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(stemmed_tokens)

In [9]:
#a function to clean the documents
def clean(text):
   text = re.sub(r"http\S+", " ", text) # remove urls
   text = re.sub(r"RT ", " ", text) # remove rt
   text = re.sub(r"@[\w]*", " ", text) # remove handles
   text = re.sub(r"[\.\,\#_\|\:\?\?\/\=]", " ", text) # remove special characters
   text = re.sub(r'\t', ' ', text) # remove tabs
   text = re.sub(r'\n', ' ', text) # remove line jump
   text = re.sub(r"\s+", " ", text) # remove extra white space
   text = text.strip()
   return text
data['processed_text'] = data['processed_text'].apply(clean)
data

,ID,text,Emotion,processed_text
0,0,i seriously hate one subject to death but now i feel reluctant to drop it,hate,seriously hate one subject death feel reluctant drop
1,1,im so full of life i feel appalled,neutral,im full life feel appalled
2,2,i sit here to write i start to dig out my feelings and i think that i am afraid to accept the possibility that he might not make it,neutral,sit write start dig feelings think afraid accept possibility might make
3,3,ive been really angry with r and i feel like an idiot for trusting him in the first place,anger,ive really angry r feel like idiot trusting first place
4,4,i feel suspicious if there is no one outside like the rapture has happened or something,neutral,feel suspicious one outside like rapture happened something
...,...,...,...,...
83677,83677,i began to feel pretty crappy,neutral,began feel pretty crappy
83678,83678,i say so because i have always considered uncle paul as that darling uncle with a knack for telling jokes exciting smiles and laughter and for mak...,fun,say always considered uncle paul darling uncle knack telling jokes exciting smiles laughter making people around feel joyful
83679,83679,i feel lonely and unloved even though i was the one that ended the relationship,love,feel lonely unloved even though one ended relationship
83680,83680,i have a sinking feeling there are more than a few people who have entertained the same heretical thought that i have,neutral,sinking feeling people entertained heretical thought


In [10]:
data['processed_text']= data['processed_text'].apply(Steem_text)
data

,ID,text,Emotion,processed_text
0,0,i seriously hate one subject to death but now i feel reluctant to drop it,hate,serious hate one subject death feel reluct drop
1,1,im so full of life i feel appalled,neutral,im full life feel appal
2,2,i sit here to write i start to dig out my feelings and i think that i am afraid to accept the possibility that he might not make it,neutral,sit write start dig feel think afraid accept possibl might make
3,3,ive been really angry with r and i feel like an idiot for trusting him in the first place,anger,ive realli angri r feel like idiot trust first place
4,4,i feel suspicious if there is no one outside like the rapture has happened or something,neutral,feel suspici one outsid like raptur happen someth
...,...,...,...,...
83677,83677,i began to feel pretty crappy,neutral,began feel pretti crappi
83678,83678,i say so because i have always considered uncle paul as that darling uncle with a knack for telling jokes exciting smiles and laughter and for mak...,fun,say alway consid uncl paul darl uncl knack tell joke excit smile laughter make peopl around feel joy
83679,83679,i feel lonely and unloved even though i was the one that ended the relationship,love,feel lone unlov even though one end relationship
83680,83680,i have a sinking feeling there are more than a few people who have entertained the same heretical thought that i have,neutral,sink feel peopl entertain heret thought


In [11]:
data['docno'] = data['ID'].astype(str)
data

,ID,text,Emotion,processed_text,docno
0,0,i seriously hate one subject to death but now i feel reluctant to drop it,hate,serious hate one subject death feel reluct drop,0
1,1,im so full of life i feel appalled,neutral,im full life feel appal,1
2,2,i sit here to write i start to dig out my feelings and i think that i am afraid to accept the possibility that he might not make it,neutral,sit write start dig feel think afraid accept possibl might make,2
3,3,ive been really angry with r and i feel like an idiot for trusting him in the first place,anger,ive realli angri r feel like idiot trust first place,3
4,4,i feel suspicious if there is no one outside like the rapture has happened or something,neutral,feel suspici one outsid like raptur happen someth,4
...,...,...,...,...,...
83677,83677,i began to feel pretty crappy,neutral,began feel pretti crappi,83677
83678,83678,i say so because i have always considered uncle paul as that darling uncle with a knack for telling jokes exciting smiles and laughter and for mak...,fun,say alway consid uncl paul darl uncl knack tell joke excit smile laughter make peopl around feel joy,83678
83679,83679,i feel lonely and unloved even though i was the one that ended the relationship,love,feel lone unlov even though one end relationship,83679
83680,83680,i have a sinking feeling there are more than a few people who have entertained the same heretical thought that i have,neutral,sink feel peopl entertain heret thought,83680


In [12]:
indexer = pt.DFIndexer("./DatasetIndex", overwrite=True)
index_ref = indexer.index(data["processed_text"], data["docno"])
print(index_ref.toString())

18:00:34.880 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (5965) - further warnings are suppressed
18:01:24.887 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 2 empty documents
./DatasetIndex/data.properties


In [13]:
#we will first load the index
index = pt.IndexFactory.of(index_ref)
#we will call getCollectionStatistics() to check the stats
print(index.getCollectionStatistics().toString())

Number of documents: 83682
Number of terms: 22630
Number of postings: 678835
Number of fields: 0
Number of tokens: 706618
Field names: []
Positions:   false



In [14]:
for kv in index.getLexicon():
  print("%s -> %s " % (kv.getKey(), kv.getValue().toString()))

Streaming output truncated to the last 5000 lines.
sharpish -> term22180 Nt=1 TF=1 maxTF=1 @{0 912983 7} 
sharpli -> term7622 Nt=3 TF=3 maxTF=1 @{0 912988 1} 
shat -> term18733 Nt=2 TF=2 maxTF=1 @{0 912998 5} 
shatp -> term19853 Nt=1 TF=1 maxTF=1 @{0 913006 3} 
shatter -> term4149 Nt=25 TF=25 maxTF=1 @{0 913010 3} 
shatteringli -> term9796 Nt=1 TF=1 maxTF=1 @{0 913083 1} 
shaun -> term973 Nt=4 TF=4 maxTF=1 @{0 913086 5} 
shauna -> term11954 Nt=2 TF=2 maxTF=1 @{0 913099 5} 
shave -> term4560 Nt=17 TF=17 maxTF=1 @{0 913107 3} 
shaven -> term11133 Nt=1 TF=1 maxTF=1 @{0 913158 5} 
shaver -> term8169 Nt=1 TF=1 maxTF=1 @{0 913162 3} 
shaw -> term17261 Nt=3 TF=3 maxTF=1 @{0 913165 7} 
shawl -> term12427 Nt=4 TF=4 maxTF=1 @{0 913176 7} 
shaz -> term21604 Nt=1 TF=1 maxTF=1 @{0 913190 5} 
shea -> term15444 Nt=2 TF=2 maxTF=1 @{0 913194 7} 
shebai -> term21329 Nt=1 TF=1 maxTF=1 @{0 913202 5} 
shebang -> term9453 Nt=1 TF=1 maxTF=1 @{0 913206 7} 
shed -> term1967 Nt=35 TF=35 maxTF=1 @{0 913210 3} 
s

In [15]:
index.getLexicon()["result"].getDocumentFrequency()

243

In [16]:
x = index.getDirectIndex()
y = index.getDocumentIndex()
z = index.getLexicon()
a = 0
for posting in x.getPostings(y.getDocumentEntry(a)):
    termid = posting.getId()
    B = z.getLexiconEntry(termid)
    print("%s with frequency %d" % (B.getKey(),posting.getFrequency()))

hate with frequency 1
reluct with frequency 1
subject with frequency 1
death with frequency 1
drop with frequency 1
feel with frequency 1


In [17]:
def preprocess(sentence):
    sentence = remove_stopwords(sentence)
    sentence = clean(sentence)
    sentence = Steem_text(sentence)
    return sentence

In [18]:
# Preprocess user query
query = "fun"
query = preprocess(query)

Tokens are: ['fun'] 



In [19]:
# Set up retrieval model with TF-IDF as weighting model
tfidf_retr = pt.BatchRetrieve(index, controls={"wmodel": "TF_IDF"}, num_results=10)

# Search for relevant documents using the retrieval model
results = tfidf_retr.search(query)
results

,qid,docid,docno,rank,score,query
0,1,6419,6419,0,6.015314,fun
1,1,48097,48097,1,5.966809,fun
2,1,839,839,2,5.643929,fun
3,1,15038,15038,3,5.643929,fun
4,1,26580,26580,4,5.643929,fun
5,1,31284,31284,5,5.643929,fun
6,1,36855,36855,6,5.643929,fun
7,1,55225,55225,7,5.643929,fun
8,1,75610,75610,8,5.643929,fun
9,1,36971,36971,9,5.611828,fun


In [20]:
# Retrieve top relevant documents based on the query
top_answer = data[data["docno"].isin(['6419', '48097'])]['processed_text']
top_answer

6419                                                                     fun feel passion ever
48097    go back energi want feel take bodi amaz fun thought fun solut fun activ movement come
Name: processed_text, dtype: object

In [21]:
# Calculate frequency of the term 'empty' in the top related documents
freq_of_items = []
for i in range(len(top_answer)):
    tweet = top_answer.iloc[i].split()
    counter = 0
    for word_in_tweet in tweet:
        if word_in_tweet == 'fun':
            counter += 1
    freq_of_items.append(counter)
print('The frequency of the item in the top related docs are:', freq_of_items)

The frequency of the item in the top related docs are: [1, 3]


In [22]:
index = pt.IndexFactory.of(index_ref)

In [23]:
# Define our retrieval model
bm25 = pt.BatchRetrieve(index, wmodel="BM25",num_results=10)
results = bm25.search(query)
results

,qid,docid,docno,rank,score,query
0,1,6419,6419,0,11.010702,fun
1,1,48097,48097,1,10.921917,fun
2,1,839,839,2,10.330903,fun
3,1,15038,15038,3,10.330903,fun
4,1,26580,26580,4,10.330903,fun
5,1,31284,31284,5,10.330903,fun
6,1,36855,36855,6,10.330903,fun
7,1,55225,55225,7,10.330903,fun
8,1,75610,75610,8,10.330903,fun
9,1,36971,36971,9,10.272143,fun


In [24]:
data[['text']][data['docno'].isin(results['docno'].loc[0:4].tolist())]

,text
839,i was feeling so carefree and wanted to go and have some fun
6419,i have fun i feel as passionate about it as ever
15038,i started feeling rotten it was a lot of fun
26580,i am hoping that they feel welcomed and that it is fun
48097,i go back to the energy that i want to feel take it into my body and then some amazing fun thought fun solution fun activity or movement comes to me


In [25]:
rm3_expander = pt.rewrite.RM3(index, fb_terms=10, fb_docs=100)

rm3_qe = bm25 >> rm3_expander
expanded_query = rm3_qe.search(query).iloc[0]["query"]

expanded_query

'applypipeline:off hope^0.017808957 carefr^0.053426865 made^0.017808957 relax^0.017808957 fun^0.775607109 want^0.040366963 welcom^0.017808957 wear^0.017808957 passion^0.023745274 went^0.017808957'

In [26]:
# Just print the expanded query with term scores
for s in expanded_query.split()[1:]:
  print(s)

print("\n" + query)

hope^0.017808957
carefr^0.053426865
made^0.017808957
relax^0.017808957
fun^0.775607109
want^0.040366963
welcom^0.017808957
wear^0.017808957
passion^0.023745274
went^0.017808957

fun


In [27]:
# After that you can search using the expanded query
expanded_query_formatted = ' '.join(expanded_query.split()[1:])

results_wqe = bm25.search(expanded_query_formatted)

print("   Before Expansion    After Expansion")
print(pd.concat([results[['docid','score']][0:5].add_suffix('_1'),
            results_wqe[['docid','score']][0:5].add_suffix('_2')], axis=1).fillna(''))


#Let's check the tweets text for the top 5 retrieved tweets
data[['text']][data['docno'].isin(results_wqe['docno'].loc[0:5].tolist())]

   Before Expansion    After Expansion
   docid_1    score_1  docid_2    score_2
0     6419  11.010702      839  11.478396
1    48097  10.921917    31284  11.478396
2      839  10.330903     6419  11.342371
3    15038  10.330903    36855  11.160740
4    26580  10.330903    48097  11.111095


,text
839,i was feeling so carefree and wanted to go and have some fun
6419,i have fun i feel as passionate about it as ever
31284,i was feeling so carefree and wanted to go and have some fun
36855,i feel carefree and fun this weekend
48097,i go back to the energy that i want to feel take it into my body and then some amazing fun thought fun solution fun activity or movement comes to me
75610,i went into it feeling much more relaxed and just out to have fun


In [28]:
!pip install flask_ngrok
!pip install pyngrok

In [29]:
df2 = data.head(50)

df2 = df2.to_dict()

df2

{'ID': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11,
  12: 12,
  13: 13,
  14: 14,
  15: 15,
  16: 16,
  17: 17,
  18: 18,
  19: 19,
  20: 20,
  21: 21,
  22: 22,
  23: 23,
  24: 24,
  25: 25,
  26: 26,
  27: 27,
  28: 28,
  29: 29,
  30: 30,
  31: 31,
  32: 32,
  33: 33,
  34: 34,
  35: 35,
  36: 36,
  37: 37,
  38: 38,
  39: 39,
  40: 40,
  41: 41,
  42: 42,
  43: 43,
  44: 44,
  45: 45,
  46: 46,
  47: 47,
  48: 48,
  49: 49},
 'text': {0: 'i seriously hate one subject to death but now i feel reluctant to drop it',
  1: 'im so full of life i feel appalled',
  2: 'i sit here to write i start to dig out my feelings and i think that i am afraid to accept the possibility that he might not make it',
  3: 'ive been really angry with r and i feel like an idiot for trusting him in the first place',
  4: 'i feel suspicious if there is no one outside like the rapture has happened or something',
  5: 'i feel jealous becasue i wanted that kin

In [30]:
def sui(df2 , que):
 i = 0

 quer = preprocess(que)

 docs_id = []

 for key, value in df2.items():
   if key == 'processed_text':
         val = value.values()
         for doc in val:
           terms = doc.split()
           for term in terms:
             if term == quer and i not in docs_id:
               docs_id.append(f'''Document number {i} -----> \n{data["text"][i]}''')
           i = i + 1
 return docs_id

In [31]:
query2 = "like"
X = sui(df2 , query2)
X

Tokens are: ['like'] 



['Document number 3 -----> \nive been really angry with r and i feel like an idiot for trusting him in the first place',
 'Document number 4 -----> \ni feel suspicious if there is no one outside like the rapture has happened or something',
 'Document number 9 -----> \ni feel like i ve regained another vital part of my life which is living',
 'Document number 10 -----> \ni feel a bit like franz liebkind in the producers not many people know it but the fuhrer was a terrific dancer',
 'Document number 14 -----> \ni just try not to talk to anyone when i feel irritable like that',
 'Document number 15 -----> \ni feel like they hated me since then',
 'Document number 19 -----> \ni feel like i have to redeem myself even though i think they realized why i was distraught and were ok with it',
 'Document number 33 -----> \ni end up getting unwanted attention from boys i want little to do with or ill be sort of starting something with a boy then find myself flirtiing with others in his presence o

In [32]:
from google.colab.output import eval_js
print (eval_js("google.colab.kernel.proxyPort(5000)"))

https://0l93vppko4z-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
from flask import Flask, request
from flask_ngrok import run_with_ngrok

# Assuming you've already defined the sui function and imported necessary modules

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
    return """
    <style>
        body {
            background-color: 	#f1c232;
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
        }

        .header {
            background-color:		#f1c232;
            color: black ;
            padding: 20px 0;

        }

        .container {
            text-align: center;
        }

        h1 {
            text-align: center;
            margin:0;
            padding: 10px 0;
        }

        #searchInput {
            padding: 10px;
            border: 1px solid #ccc;
            border-radius: 50px; /* Increased border-radius for a rounded appearance */
            margin-bottom: 10px;
            width: 300px; /* Adjust the width as needed */
            box-sizing: border-box; /* Include padding and border in the element's total width */
            transition: border-color 0.5s; /* Smooth transition for border color change */
        }

        #searchInput:focus {
            border-color: #403072; /* Change border color on focus */
        }

        button {
            padding: 10px 20px;
            background-color:#ce7e00;
            color: white;
            border: none;
            border-radius: 20px; /* Increased border-radius for a rounded appearance */
            cursor: pointer;
            transition: background-color 0.5s; /* Smooth transition for background color change */
        }

        button:hover {
            background-color: #7d73af; /* Change background color on hover */
        }

          #searchResult {
            text-align: 10 px; /* Center the search results */
            color: black; /* Set the color of search results */
            margin: 10 px;
            padding: 10px
        }
    </style>

    <div class="header">
        <h1>Search Engine</h1>
    </div>
    <div class="container">
        <input type="text" id="searchInput" placeholder="search...">
        <button onclick="search()">Search</button>
    </div>
    <div id="searchResult"></div>

    <script>
        function search() {
            var searchTerm = document.getElementById("searchInput").value;
            fetch('/search', {
                method: 'POST',
                body: JSON.stringify({ query: searchTerm }),
                headers:{
                    'Content-Type': 'application/json'
                }
            })
            .then(response => response.json())
            .then(data => {
                console.log("Received data:", data); // Debug: Check if data is received
                var resultDiv = document.getElementById("searchResult");
                resultDiv.innerHTML = "<h2>Relevant Documents IDs:</h2>";
                if (data.results.length === 0) {
                    resultDiv.innerHTML += "<p>No documents found</p>";
                } else {
                    data.results.forEach(doc => {
                        console.log("Displaying document:", doc); // Debug: Check if document is displayed
                        resultDiv.innerHTML += "<p>" + doc + "</p>";
                    });
                }
            })
            .catch(error => {
                console.error('Error occurred during fetch:', error); // Debug: Log fetch errors
            });
        }
    </script>
    """

@app.route("/search", methods=['POST'])
def search():
    query = request.json['query']
    print("Received query:", query)  # Debug: Check if Flask receives the query
    results = sui(df2, query)
    print("Search results:", results)  # Debug: Check if sui function returns results
    return {'results': results}

app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-17:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 715, in urlopen
    httplib_response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packag

Received query: happy
Tokens are: ['happy'] 

Search results: ['Document number 34 -----> \ni just couldnt help feeling a little bit bitter towards his great big happy grin']


INFO:werkzeug:127.0.0.1 - - [11/May/2024 18:11:01] "POST /search HTTP/1.1" 200 -


Received query: anger
Tokens are: ['anger'] 

Search results: []


...

Received query: hate
Tokens are: ['hate'] 

Search results: ['Document number 0 -----> \ni seriously hate one subject to death but now i feel reluctant to drop it', 'Document number 15 -----> \ni feel like they hated me since then', 'Document number 42 -----> \ni feel and some is just a hateful of hollow yes i hear many smiths these days']


INFO:werkzeug:127.0.0.1 - - [11/May/2024 18:11:42] "POST /search HTTP/1.1" 200 -


Received query: nutral
Tokens are: ['nutral'] 

Search results: []


INFO:werkzeug:127.0.0.1 - - [11/May/2024 18:14:30] "POST /search HTTP/1.1" 200 -


Received query: feel
Tokens are: ['feel'] 

Search results: ['Document number 0 -----> \ni seriously hate one subject to death but now i feel reluctant to drop it', 'Document number 1 -----> \nim so full of life i feel appalled', 'Document number 2 -----> \ni sit here to write i start to dig out my feelings and i think that i am afraid to accept the possibility that he might not make it', 'Document number 3 -----> \nive been really angry with r and i feel like an idiot for trusting him in the first place', 'Document number 4 -----> \ni feel suspicious if there is no one outside like the rapture has happened or something', 'Document number 5 -----> \ni feel jealous becasue i wanted that kind of love the true connection between two souls and i wanted that', 'Document number 7 -----> \ni finally fell asleep feeling angry useless and still full of anxiety', 'Document number 8 -----> \ni feel a bit annoyed and antsy in a good way', 'Document number 9 -----> \ni feel like i ve regained anoth

In [ ]:
vaswani_dataset = pt.datasets.get_dataset("vaswani")

df= vaswani_dataset.get_topics()

data['docno'] = df.index

# Rename column 'A' to 'X'
df = df.rename(columns={'query': 'Text'})

qrels = vaswani_dataset.get_qrels()

qrels['docno']=qrels['docno'].astype(str)

df

In [ ]:
indexref2 = vaswani_dataset.get_index()
index2 = pt.IndexFactory.of(indexref2)

print(index2.getCollectionStatistics().toString())

In [ ]:
retr = pt.BatchRetrieve(index2, controls = {"wmodel": "TF_IDF"})

res = retr.search("mathematical")
res

In [ ]:
eval = pt.Evaluate(res,qrels)
eval